# SITCOMTN-058: Stars for CWFS correction 

A catalog of bright stars with clean surrounding is required to perform focus and alignment with the AuxTel.
It is anticipated that rebuilding star catalog is required when new criteria should be applied to the selection of stars. This technote is to describe the notebook that creates a new catalog based on queries from Tycho-2 and HD catalog. This notebook can be found on this github [repo](https://github.com/lsst-sitcom/sitcomtn-058/tree/main/_static).
**Please note that it is recommended to run this notebook in a local environment as it may take several tens of minutes or longer to complete.**

https://sitcomtn-058.lsst.io/
https://jira.lsstcorp.org/browse/SITCOM-616

# 1. Create a notebook with the Tycho-2 catalogue

Create a notebook that starts with the HD catalogue, then does the large elimination of sources via magnitude (6-8) and observability from the southern hemisphere. <br>

- Catalog is imported from the Tycho-2 catalogue (Hog+00)(Vizier I/259/tyc2; http://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/259/tyc2) <br>

In [ ]:
#1-1 Import 
import pandas as pd
import json
from astroquery.vizier import Vizier
from astropy.table.pprint import conf
from astropy.table import  QTable, Table, Column
from astropy import coordinates
import astropy.units as u
import numpy as np
import sys, re, time, math
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
import os.path              
from astroquery.simbad import Simbad
from IPython.display import Markdown as md

pd.options.display.max_rows = 10

In [ ]:
#1-2 Define the timeout or limit for Simbad query
conf.max_lines = -1
conf.max_width = -1
Vizier.ROW_LIMIT = -1
Simbad.TIMEOUT = 1200
customSimbad = Simbad()
customSimbad.TIMEOUT = 1200

The file name ('file') can be changed to what you want to save as. <br>
At the first time, query the Vizier catalog and save the list of all stars with constraints into json file. If the file already exists in local, it reads the existing file. <br>

- <b>``RAmdeg``: String (unit: deg)</b><br>
    Mean Right Asc, ICRS, epoch=J2000 with logical operator <br>
- <b>``DEmdeg``: String (unit: deg)</b><br>
    Mean Decl, ICRS, at epoch=J2000 with logical operator <br>
- <b> ``VTmag``: String (unit: mag)</b><br>
    Tycho-2 VT magnitude with logical operator <br>

In [ ]:
DEmdeg = '<10.0' ; VTmag = '<10'

In [ ]:
file = './cwfs_tycho2_stars.pd' 

In [ ]:
if os.path.exists(file):
    print('Read existing Vizier Catalog in Local'+file)
    tycho_star_all = pd.read_json(file)
else :
    print('Read Vizier Catalog I/259/tyc2 ')
    tycho_star_all = Vizier.query_constraints(catalog='I/259/tyc2', DEmdeg=DEmdeg, VTmag=VTmag)[0]
    tycho_star_all = tycho_star_all.to_pandas()
    result = tycho_star_all.to_json(file)

# 2. Cutting using the Criteria 

  Following is the selection criteria to cut sample. 
- Criteria 1: Trim targets which have contaminants < 10th magnitude within 1 arcminute <br>
- Criteria 2: targets which have bright stars (mag<6) within 7 arcminutes <br>
- Criteria 3: Exclude targets which have bright stars (mag<3) within 20 arcminutes <br>
 
 You can also change the following parameters:
 - <b>``mag_upper`` & ``mag_lower``</b>: 
   the upper and lower limit of the CWFS stars. The default values are V_upper = 6 mag, V_lower = 8 mag. <br>
 - <b>``pm_cut``</b>: the upper limit of net proper motion (Default: +-100mas/yr) <br>
 - <b>``net_pm``</b>: net proper motion (net_pm $\equiv$ $\sqrt{(pmRA)^2+(pmDE)^2}$); <br>
 - <b>Criteria N</b>:<br> 
     - ``mag_cutN``: Magnitude cut of nearby star <br>
     - ``area_cutN``: within N arcminutes <br>

In [ ]:
#1-3 Define the cuts 
mag_upper = 6 ; mag_lower = 8; pm_cut = 100 #pm_cut for net_pm (sqrt(pmRA*cosDec^2.0+pmDEC^2.0))
area_cut1 = 1; area_cut2 = 7; area_cut3 = 20 #arcmin.
mag_cut1 = 10; mag_cut2 = 6; mag_cut3= 3 #Vmag

In [ ]:
tycho_star_all['net_pm'] = (((tycho_star_all['pmRA'])**2.0 + (tycho_star_all['pmDE'])**2.0)**0.5)
tycho_star_cut = tycho_star_all[(tycho_star_all['VTmag'] > mag_upper) & (tycho_star_all['VTmag'] < mag_lower) & (tycho_star_all['net_pm'] < pm_cut)]

In [ ]:
display(tycho_star_cut)

In [ ]:
# function: seperation between position 1(RA1, DEC1) and position 2(RA2, DEC2) on Sky
def sep(RA1,RA2,Dec1,Dec2):
    sep = ((((RA1-RA2)*np.cos(Dec1))**(2.0))+((Dec1-Dec2)**2.0))**(0.5)
    return sep

In [ ]:
tycho_star_cut_criteria = pd.DataFrame.copy(tycho_star_cut)

for i in tycho_star_cut.index:
    within_1arcmin = (sep(tycho_star_cut["RA_ICRS_"][i],tycho_star_all["RA_ICRS_"],tycho_star_cut["DE_ICRS_"][i],tycho_star_all["DE_ICRS_"]) < area_cut1/60.0) & (tycho_star_all["VTmag"] < mag_cut1)
    within_7arcmin = (sep(tycho_star_cut["RA_ICRS_"][i],tycho_star_all["RA_ICRS_"],tycho_star_cut["DE_ICRS_"][i],tycho_star_all["DE_ICRS_"]) < area_cut2/60.0) & (tycho_star_all["VTmag"] < mag_cut2)
    within_20arcmin = (sep(tycho_star_cut["RA_ICRS_"][i],tycho_star_all["RA_ICRS_"],tycho_star_cut["DE_ICRS_"][i],tycho_star_all["DE_ICRS_"]) < area_cut3/60.0)& (tycho_star_all["VTmag"] < mag_cut3)
    if np.count_nonzero(within_1arcmin) > 1 or np.count_nonzero(within_7arcmin) > 0 or np.count_nonzero(within_20arcmin) > 0:
        tycho_star_cut_criteria.drop([i], inplace=True)

In [ ]:
display(tycho_star_cut_criteria)

Now we will match this catalog with HD identifications for Tycho-2 stars (Fabricius+, 2002), queried from Vizier 'IV/25/tyc2_hd' (http://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=IV/25/tyc2_hd). 

In [ ]:
print('Query Vizier Catalog IV/25/tyc2_hd to match TYC ID and HD ID')
HD_stars_all = Vizier.query_constraints(catalog='IV/25/tyc2_hd')[0]

In [ ]:
index_only = HD_stars_all[0:0]
index_only.add_columns([(),(),()],names=['pmRA','pmDE','net_pm'])
index_only['pmRA'] = index_only['pmRA'].astype(np.float32)
index_only['pmDE'] = index_only['pmDE'].astype(np.float32)
index_only['net_pm'] = index_only['net_pm'].astype(np.float32)
HD_star_match= Table(index_only)

In [ ]:
for i in tycho_star_cut_criteria.index: 
    condition = (HD_stars_all["TYC1"]==tycho_star_cut_criteria["TYC1"][i]) & \
    (HD_stars_all["TYC2"]==tycho_star_cut_criteria['TYC2'][i]) & \
    (HD_stars_all["TYC3"]==tycho_star_cut_criteria['TYC3'][i])
    if np.count_nonzero(condition) == 1:
        table = HD_stars_all[condition]
        table['pmRA'] = tycho_star_cut_criteria["pmRA"][i] 
        table['pmDE'] = tycho_star_cut_criteria["pmDE"][i] 
        table['net_pm'] = tycho_star_cut_criteria["net_pm"][i]
        HD_star_match.add_row(table[:][0])

In [ ]:
HD_star_match.show_in_notebook(display_length=10)

Then, query simbad data for each selected CWFS stars:<br>
- The default VOTable fields: ``MAIN_ID``, ``RA``, ``DEC``, ``RA_PREC``, ``DEC_PREC``, ``COO_ERR_MAJA``, ``COO_ERR_MINA``, ``COO_ERR_ANGLE``, ``COO_QUAL``, ``COO_WAVELENGTH``, ``COO_BIBCODE``, ``SCRIPT_NUMBER_ID`` <br>
- Add ``flux_name(V)``, ``flux(V)``, ``flux_error(V)``, ``flux_system(V)``, ``flux_bibcode(V)``, ``flux_qual(V)``, ``flux_univ(V)`` fields for VOTable.<br> 
- Note that some stars can have non-HD Main ID in the final catalog, but these stars also have HD ID. 

In [ ]:
customSimbad = Simbad()
customSimbad.add_votable_fields('flux_name(V)', 'flux(V)', 'flux_error(V)',\
                                'flux_system(V)','flux_bibcode(V)', 'flux_qual(V)', 'flux_unit(V)')

for i in range(0,len(HD_star_match["HD"])):
    result_table = customSimbad.query_object('HD '+str(HD_star_match["HD"][i]))
    result_table["HD_ID"] = 'HD '+str(HD_star_match["HD"][i])
    result_table["pmRA"] = HD_star_match["pmRA"][i]
    result_table["pmDE"] = HD_star_match["pmDE"][i]
    result_table["net_pm"] = HD_star_match["net_pm"][i]
    if i==0:
        final = result_table
    else:
        final.add_row(result_table[:][0])

In [ ]:
final.show_in_notebook(display_length=10)

If you want to chech all available fields for VOTable, please run the following command.

In [ ]:
customSimbad.list_votable_fields()

# 3. Manually Exclude the stars from the list (Optional)
This section is to exclude the stars from the list manually. Put the ID (HD) of the stars on the ``Remove_main_id`` list below. Even if there are any stars not included in the final table, it is fine to run. If it is not nessary to remove any star, please skip this step. 

In [ ]:
Remove_main_id = ["HD22746","HD452"] #HD NNNNNN 

In [ ]:
p= re.compile("\d*\.?\d+")
customSimbad = Simbad()
for i in range(len(Remove_main_id)):
    number = p.findall(Remove_main_id[i])[0]
    Remove_main_id_simbad= customSimbad.query_region('HD '+str(number))["MAIN_ID"]
    Remove = (final["MAIN_ID"] == Remove_main_id_simbad[0])
    if np.count_nonzero(Remove) !=0 :
        remove_index = [i for i, x in enumerate(Remove) if x]
        final.remove_row(remove_index[0])
        print(Remove_main_id_simbad[0]+' is now removed from the final catalog')

# 4. Save the result into the file

Export table into a json data file. The name of the output catalog can be changed using ``file_name_final``.

In [ ]:
file_name_final = 'HD_cwfs_stars.pd' #file_name
result = final.to_pandas().to_json(file_name_final)
print('List of Stars was exported to '+file_name_final)

# 5. Plot for the distribution of the Stars on Sky. 
This section is to check the distribution of the final selection of CWFS stars. <br>
If the catalog already exists, you can only plot the distribution. ``plot_mwd``is adopted from http://balbuceosastropy.blogspot.com/2013/09/the-mollweide-projection.html. 

In [ ]:
file_name_final = 'HD_cwfs_stars.pd' #file_name
final_list = pd.read_json(file_name_final)

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import ephem # to make coordinate systems conversions
from IPython.core.display import HTML # To include images as HTML

In [ ]:
def plot_mwd(RA,Dec,org=0,title='Mollweide projection', projection='mollweide', color="blue"):
    ''' RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    '''
    x = np.remainder(RA+360-org,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    # reverse the scale: East to the left
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org,360)
    fig = plt.figure(figsize=(10, 5))
    #ax = fig.add_subplot(111, projection=projection, axisbg ='LightCyan')

    ax = fig.add_subplot(111, projection=projection)
    ax.scatter(np.radians(x),np.radians(Dec), c=color, marker='.', s=1)  # convert degrees to radians
    ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
    ax.set_title(title)
    ax.title.set_fontsize(15)
    ax.set_xlabel("RA")
    ax.xaxis.label.set_fontsize(12)
    ax.set_ylabel("Dec")
    ax.yaxis.label.set_fontsize(12)
    ax.grid(True)

In [ ]:
c = SkyCoord(final_list["RA"], final_list["DEC"], unit=(u.hourangle, u.deg))
plot_mwd(c.ra.deg,c.dec.deg, title="Distribution of CWFS Stars")

plt.show()

# Appendix. Check the Field of the Individual Star
If it is necessary to check the FOV of the individual star, you can check it manually.<br>
The default FOV of the image are 6.7' x 6.7'. 

In [ ]:
star_name_img_query = "HD 2527"
FOV=6.7*1/60.0 #6.7 x 6.7 arcminutes for AuxTel

In [ ]:
from astroquery.skyview import SkyView
import numpy as np
survey_name = ["DSS2 Blue", "DSS2 Red", "DSS2 IR"]
img = SkyView.get_images(star_name_img_query,survey=survey_name,\
                         height=FOV*u.degree,width=FOV*u.degree,coordinates='J2000',grid=True,gridlabels=True)

In [ ]:
ncol=len(survey_name)
fig,ax = plt.subplots(ncols=ncol,figsize=(24,8))
for i in range(ncol):
    plot = ax[i].imshow(img[i][0].data,vmax=np.max(img[i][0].data)*.95,\
                        vmin=np.max(img[i][0].data)*.25, aspect='equal')
    ax[i].set_title(str(survey_name[i]), fontsize=15)
    fig.gca().invert_yaxis()

print(star_name_img_query, 'FOV = '+str((FOV*60))+'\"'+'x '+str((FOV*60))+'\"')